In [ ]:
import pandas as pd
import sqlite3

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
db_path = "db.sqlite3"

try:
    conn = sqlite3.connect(db_path)

    query = """
        SELECT
            oi.order_id AS "Order ID",
            o.order_datetime AS "Order Datetime",
            o.order_price AS "Order Price",
            oi.product_id AS "Product ID",
            p.product_name AS "Product Name",
            p.price AS "Product Price",
            oi.quantity AS "Quantity"
        FROM order_items AS oi
        JOIN orders AS o ON oi.order_id = o.id
        JOIN products AS p ON oi.product_id = p.id
    """

    df = pd.read_sql_query(query, conn)

    print(f"Number of rows in final dataframe: {len(df)}")
    print("First 5 rows of merged dataframe:")
    print(df.head())

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    if 'conn' in locals():
        conn.close()


# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
quantity_counts = df.groupby('Product Name')['Quantity'].sum().sort_values(ascending=False)
top_10_quantity = quantity_counts.head(10)

total_quantity = quantity_counts.sum()
others_quantity_count = total_quantity - top_10_quantity.sum()

pie_data_quantity = pd.concat([
    top_10_quantity.rename("Count"),
    pd.Series(others_quantity_count, index=['OTHERS']).rename("Count")
], axis=0).to_frame()

pie_data_quantity['Percentage (%)'] = (pie_data_quantity['Count'] / total_quantity * 100).round(2)
pie_data_quantity['Count'] = pie_data_quantity['Count'].astype(int)

print("### Task 1: Top 10 most popular products (by QUANTITY) ###")
print(pie_data_quantity, "\n")


# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df['Item Price'] = df['Product Price'] * df['Quantity']

revenue_by_product = df.groupby('Product Name')['Item Price'].sum().sort_values(ascending=False)
top_10_revenue = revenue_by_product.head(10)

total_revenue = revenue_by_product.sum()
others_revenue_count = total_revenue - top_10_revenue.sum()

pie_data_revenue = pd.concat([
    top_10_revenue.rename("Revenue"),
    pd.Series(others_revenue_count, index=['OTHERS']).rename("Revenue")
], axis=0).to_frame()

pie_data_revenue['Percentage (%)'] = (pie_data_revenue['Revenue'] / total_revenue * 100).round(2)
pie_data_revenue['Revenue'] = pie_data_revenue['Revenue'].round(2)

print("### Task 2: Top 10 Most Profitable Products (by REVENUE) ###")
print(pie_data_revenue, "\n")


# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df['Order Datetime'] = pd.to_datetime(df['Order Datetime'])

unique_orders = df[['Order ID', 'Order Datetime', 'Order Price']].drop_duplicates()

unique_orders['Order Hour'] = unique_orders['Order Datetime'].dt.hour

hourly_income_total = unique_orders.groupby('Order Hour')['Order Price'].sum().round(2)

full_hours = pd.Series(0.0, index=range(24))
hourly_income_total = hourly_income_total.combine_first(full_hours).sort_index().rename('Total Income')

print("### Task 3: Total Hourly Income (Order Hour 0-23) ###")
print(hourly_income_total.to_frame(), "\n")

def create_bar_chart_ascii(series, title) -> str:
    max_val = series.max()
    output = [f"--- {title} ---"]
    for index, value in series.items():
        bar_length = int((value / max_val) * 40) if max_val != 0 else 0
        bar = "#" * bar_length
        output.append(f"{str(index):<10}: | {bar:<40} {value:,.2f}")
    return "\n".join(output)

print(create_bar_chart_ascii(hourly_income_total, "Hourly Income"), "\n")


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

unique_orders = df[['Order ID', 'Order Datetime', 'Order Price']].drop_duplicates()

unique_orders['Order Day Of The Week'] = unique_orders['Order Datetime'].dt.day_name().astype(
    pd.CategoricalDtype(categories=day_order, ordered=True)
)

daily_income_total = (
    unique_orders.groupby('Order Day Of The Week')['Order Price']
    .sum()
    .round(2)
    .sort_index()
    .rename('Total Income')
)

print("### Task 4: Total Income by Day of the Week ###")
print(daily_income_total.to_frame(), "\n")

print(create_bar_chart_ascii(daily_income_total, "Income by Day of the Week"))
